### Problem Statement:
### This assignment aims to utilize Long Short-Term Memory (LSTM) algorithms to perform next word prediction on a given dataset. Next word prediction involves predicting the next word in a sequence of text based on the preceding words. By implementing LSTM (Long Short-Term Memory) networks, to learn how to process sequential data and capture long-term dependencies for predicting the next word in a sentence.

In [1]:
!pip install --upgrade pip
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: keras
    Found existing installation: keras 3.1.1
    Uninstalling keras-3.1.1:
      Successfully uninstalled keras-3.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-nlp 0.8.2 requires keras-core, which is not installed.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


#### The code provided bellow : Loads the text file and reads its content. Displaying the first 1000 characters can be helpful for verifying if the file is loaded correctly.

In [2]:
# Specify the path to your text file
file = open('/kaggle/input/lstm-data/LSTM DATA.txt', encoding='utf8')

#This line of code opens the specified text file 
# The encoding as UTF-8, which is commonly used for handling text files containing a variety of characters from different languages

In [3]:
lines = file.readlines()

# Close the files
file.close()

In [4]:
lines [:5]

['\ufeffThe Project Gutenberg eBook of Pride and Prejudice\n',
 '    \n',
 'This ebook is for the use of anyone anywhere in the United States and\n',
 'most other parts of the world at no cost and with almost no restrictions\n',
 'whatsoever. You may copy it, give it away or re-use it under the terms\n']

In [5]:
# Join the lines into a single string
df = ' '.join(lines)

# Step 1: Removing unnecessary information
start_index = df.find("The Project Gutenberg eBook of Pride and Prejudice")
df = df[start_index:]


In [6]:
# Remove unwanted characters
df = df.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“', '').replace('”', '')

In [7]:
df[:500]

'The Project Gutenberg eBook of Pride and Prejudice      This ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook.  '

#### Preprocessing the text data for further analysis. Let me break down what each step does:

   #### 1) Removing unnecessary information: This step finds the index of a specific string ("The Project Gutenberg eBook of Pride and Prejudice") and removes everything before it. This could be useful if you want to remove any metadata or headers from your text.

   #### 2) Tokenizing the text into words: This splits the text into individual words based on whitespace. This step is essential for text processing tasks.

  ####  3) Lowercasing the text: Converting all words to lowercase ensures that the model treats words like "Word" and "word" as the same, which helps in avoiding duplication.

  ####  4) Removing punctuation and special characters: This step uses regular expressions to remove any characters that are not alphanumeric (letters or numbers). It helps clean up the text and standardize it for analysis.

#### Finally, the processed text is joined back together into a single string. This preprocessed text can now be used for tasks like next-word prediction. 

In [8]:
 # import the libraries 're' for regular expressions 
import re

# Step 3: Lowercasing the text
df = df.lower() 

# Additional cleaning using regex
df = re.sub(r'\ufeff', '', df) 

df = re.sub(r'[“”"‘’]', '', df)


In [9]:
# Displaying the first 500 characters can be helpful for verifying if the file is processed correctly.¶
df[:1000]

'the project gutenberg ebook of pride and prejudice      this ebook is for the use of anyone anywhere in the united states and most other parts of the world at no cost and with almost no restrictions whatsoever. you may copy it, give it away or re-use it under the terms of the project gutenberg license included with this ebook or online at www.gutenberg.org. if you are not located in the united states, you will have to check the laws of the country where you are located before using this ebook.  title: pride and prejudice   author: jane austen  release date: june 1, 1998 [ebook #1342]                 most recently updated: april 14, 2023  language: english  credits: chuck greif and the online distributed proofreading team at http://www.pgdp.net (this file was produced from images available at the internet archive)   *** start of the project gutenberg ebook pride and prejudice ***                                 [illustration:                               george allen                  

In [10]:
# To find the length of the processed text, you can simply use the len() function. 
len(df)

743529

  ####  Tokenization: Using the Tokenizer class from Keras,initialize a tokenizer object. Specify the maximum number of words (num_words) and a set of characters to filter out (filters). In this case removing punctuation characters and special characters commonly found in text. Additionally, you set lower=True to convert all words to lowercase during tokenization.

####    Fit Tokenizer on Text: You call the fit_on_texts() method of the tokenizer, passing the preprocessed text as input. This updates the tokenizer's internal vocabulary based on the text provided.

####    Generate Word Indices: After fitting the tokenizer, you retrieve the word index, which is a dictionary mapping words to their respective integer indices.

##### This prepares your text data for further processing, such as converting text sequences to sequences of integers based on these word indices.

In [11]:
!pip install optree


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts([df])
word_index = tokenizer.word_index

In [13]:
import pickle

#### Save the tokenizer as a token.pkl gormat externally, Useing pickle libraries.
pickle.dump(tokenizer,open('token.pkl','wb'))

In [14]:
len(word_index) # Check the vocabulary size
# The length of word_index gives you the number of unique words in your preprocessed text. 
# You can obtain this length using the len() function

7172

#### This process effectively creates sequences of fixed length from the tokenized text data, which is often used as input data for models like recurrent neural networks (RNNs) or long short-term memory networks (LSTMs) in tasks like sequence prediction or language modeling.

In [15]:
# Convert text sequences to sequences of integers

input_sequences = [] # This initializes an empty list input_sequences where we will store the sequences of integers.

for sentence in df.split('. '):  # This loop iterates over each sentence in the text data. 
    # The assumption here is that df contains the text data, and split('. ') is used to split the text into sentences based on the period followed by a space, 
    # which is a common way to denote the end of a sentence. You may need to adjust this based on the structure of your text data.
    tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0] #Inside the loop, each sentence is tokenized into a sequence of integers using the texts_to_sequences method of the tokenizer object. 
    # This method converts each word in the sentence to its corresponding integer index based on the vocabulary learned during tokenization.
    
    for i in range(1, len(tokenized_sentence)):
        input_sequences.append(tokenized_sentence[:i+1])
        # For each tokenized sentence, this loop generates multiple input-output pairs for training the model. 
        # It starts from the second word (index 1) and iterates up to the length of the tokenized sentence.
        # At each iteration, it creates a sequence of integers from the beginning of the sentence up to the current word index (i+1). 
        # This sequence represents the input, and the next word after this sequence represents the output. 
        # These input-output pairs are then appended to the input_sequences list.

### After running this code, input_sequences will contain sequences of integers representing input-output pairs for training the model. Each sequence consists of an increasing number of words from the beginning of a sentence, and the final element of each sequence is the next word in the sentence. These sequences can be used to train an LSTM or similar model for next word prediction.

In [16]:
input_sequences [0:15]

[[1, 182],
 [1, 182, 391],
 [1, 182, 391, 1005],
 [1, 182, 391, 1005, 3],
 [1, 182, 391, 1005, 3, 299],
 [1, 182, 391, 1005, 3, 299, 4],
 [1, 182, 391, 1005, 3, 299, 4, 951],
 [1, 182, 391, 1005, 3, 299, 4, 951, 41],
 [1, 182, 391, 1005, 3, 299, 4, 951, 41, 1005],
 [1, 182, 391, 1005, 3, 299, 4, 951, 41, 1005, 23],
 [1, 182, 391, 1005, 3, 299, 4, 951, 41, 1005, 23, 21],
 [1, 182, 391, 1005, 3, 299, 4, 951, 41, 1005, 23, 21, 1],
 [1, 182, 391, 1005, 3, 299, 4, 951, 41, 1005, 23, 21, 1, 508],
 [1, 182, 391, 1005, 3, 299, 4, 951, 41, 1005, 23, 21, 1, 508, 3],
 [1, 182, 391, 1005, 3, 299, 4, 951, 41, 1005, 23, 21, 1, 508, 3, 549]]

In [17]:
max_len = max([len(x) for x in input_sequences])
max_len

# This code calculates the maximum length of the input sequences.
# max_len: This is the maximum length that you want the sequences to be padded or truncated to. 
# All sequences longer than maxlen will be truncated, and sequences shorter than maxlen will be padded.

152

In [18]:
# Pad the sequences This code snippet pads the input sequences to ensure that they all have the same length.
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_len, padding='pre')

In [19]:
padded_input_sequences

array([[   0,    0,    0, ...,    0,    1,  182],
       [   0,    0,    0, ...,    1,  182,  391],
       [   0,    0,    0, ...,  182,  391, 1005],
       ...,
       [   0,    0,    0, ...,    2,  230,  126],
       [   0,    0,    0, ...,  230,  126,  524],
       [   0,    0,    0, ...,  126,  524, 1753]], dtype=int32)

In [20]:
len(padded_input_sequences)

124648

In [21]:
import numpy as np
# Create predictors and labels
X = np.array([x[:-1] for x in padded_input_sequences])
y = np.array([x[-1] for x in padded_input_sequences])

# Print the shapes of X and y
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

Shape of X: (124648, 151)
Shape of y: (124648,)


In [22]:
import tensorflow as tf

# Convert y to one-hot encoding
num_words = len(word_index) + 1
y_one_hot = tf.keras.utils.to_categorical(y, num_classes=num_words)


#### In summary, using one-hot encoding for categorical labels ensures compatibility with loss functions, facilitates meaningful comparison between model predictions and ground truth labels, and helps in stable training and effective learning of neural network models, especially for classification tasks.

In [23]:
y_one_hot.shape

(124648, 7173)

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM,BatchNormalization

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, BatchNormalization, Dense

model = Sequential()
model.add(Embedding(input_dim=15000, output_dim=100, input_shape=(151,)))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(LSTM(64))
model.add(BatchNormalization())
model.add(Dense(7173, activation='softmax'))


2024-04-15 20:51:34.977364: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-15 20:51:34.977477: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-15 20:51:34.977553: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-15 20:51:34.977639: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-15 20:51:34.977707: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-15 20:51:34.977907: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-04-15 20:51:34.977983: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this 

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 151, 100)          1500000   
                                                                 
 lstm (LSTM)                 (None, 151, 256)          365568    
                                                                 
 dropout (Dropout)           (None, 151, 256)          0         
                                                                 
 lstm_1 (LSTM)               (None, 151, 128)          197120    
                                                                 
 dropout_1 (Dropout)         (None, 151, 128)          0         
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 batch_normalization (Batch  (None, 64)                2

In [27]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [28]:
# Train the model
batch_size = 64
epochs = 50

history = model.fit(X, y_one_hot, batch_size=batch_size, validation_split=0.2, epochs=epochs)

Epoch 1/50
1559/1559 [==============================] - 946s 604ms/step - loss: 6.5053 - accuracy: 0.0663 - val_loss: 5.9738 - val_accuracy: 0.0961
Epoch 2/50
1559/1559 [==============================] - 940s 603ms/step - loss: 5.5270 - accuracy: 0.1148 - val_loss: 5.7902 - val_accuracy: 0.1184
Epoch 3/50
1559/1559 [==============================] - 938s 602ms/step - loss: 5.1879 - accuracy: 0.1375 - val_loss: 5.6963 - val_accuracy: 0.1256
Epoch 4/50
1559/1559 [==============================] - 938s 602ms/step - loss: 4.9399 - accuracy: 0.1544 - val_loss: 5.6902 - val_accuracy: 0.1306
Epoch 5/50
1559/1559 [==============================] - 937s 601ms/step - loss: 4.7256 - accuracy: 0.1653 - val_loss: 5.7460 - val_accuracy: 0.1318
Epoch 6/50
1559/1559 [==============================] - 938s 602ms/step - loss: 4.5329 - accuracy: 0.1762 - val_loss: 5.8621 - val_accuracy: 0.1349
Epoch 7/50
1559/1559 [==============================] - 938s 602ms/step - loss: 4.3478 - accuracy: 0.1879 - val_

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# Plot training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
model.save('next_word_predictor.h5')

In [ ]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
import time
import numpy as np

text = "the"

for i in range(25):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=3, padding='pre')
  # predict
  predicted_index = np.argmax(model.predict(padded_token_text))
    
  for word,index in tokenizer.word_index.items():
    if index == predicted_index:
      text = text + " " + word
      print(text)
      time.sleep(2)
  

#### Tuning the Hyperparameters


In [ ]:
!pip install scikeras==0.12.0

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

# Define the LSTM model
# Define the LSTM model
def create_lstm_model():
    model.add(Embedding(input_dim=15000, output_dim=100, input_shape=(151,)))
    model.add(LSTM(256, return_sequences=True))
    model.add(Dropout(rate=0.2))
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(rate=0.2))
    model.add(LSTM(64))
    model.add(BatchNormalization())
    model.add(Dense(7173, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
# create model
model = KerasClassifier(model=create_lstm_model, verbose=0)

In [ ]:
# Define hyperparameters for grid search
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'batch_size': [64, 128, 256],
    'epochs': [50, 100],
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)

In [ ]:
grid_result = grid.fit(X,y_one_hot)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))